In [1]:
from hmmlearn import hmm

In [2]:
import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings
import pickle

In [3]:
warnings.filterwarnings("ignore")

In [4]:
from nilmtk import *
import os
import nilmtk

In [5]:
st = pd.HDFStore(os.path.expanduser("/Users/nipunbatra/wiki-15min.h5"))

In [6]:
metadata_df = pd.read_csv("/Users/nipunbatra/w/dataport-metadata.csv",index_col=0)

In [7]:
feeds = {'use':'aggregate',
        'air1':'hvac',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [8]:
cities = ['Austin']

In [9]:
months=12
start='1-1-2015'
start_month=1
end = '12-31-2015'
end_month=12

In [66]:
model = {}
for city in ['San Diego','Austin','Boulder']:
    print city
    print "*"*20
    city_data = metadata_df[metadata_df['city'] == city]
    city_homes = city_data.index.values.astype('int')
    city = city.replace(" ","")
    model[city] = {}
    for appliance in ["air1", "refrigerator1", "clotheswasher1", 
                      "dishwasher1", "microwave1", "car1","oven1","waterheater1","dryer1"]:
        print appliance, "training"
        if appliance in ["refrigerator1"]:
            activation_min = 0.05
        else:
            activation_min = 0.0
        o = []
        for home in city_homes[:50]:
            try:
                dft = st['/%d' %home][appliance][start:end]
                dft = dft.resample("15T", how="mean")
                appl_power = dft.dropna().values.reshape(-1,1)
                activation = (dft>10).sum()*1.0/len(dft)
                
            
                
                if activation>activation_min:
                    o.append(appl_power)
            except:
                pass
        if len(o)>1:
            o = np.array(o)
            mod = hmm.GaussianHMM(3, "full")
            mod.fit(o)
            model[city][feeds[appliance]] = mod
            print "Means for %s are" %appliance
            print mod.means_
        else:
            print "Not enough samples for %s" %appliance

San Diego
********************
air1 training
Means for air1 are
[[    0.        ]
 [  349.4476466 ]
 [ 2022.82781261]]
refrigerator1 training
Means for refrigerator1 are
[[  87.05974175]
 [   2.95077789]
 [ 221.86712369]]
clotheswasher1 training
Means for clotheswasher1 are
[[   2.1808973 ]
 [ 104.1043107 ]
 [  24.84099837]]
dishwasher1 training
Means for dishwasher1 are
[[   0.        ]
 [ 563.30253133]
 [  32.83688517]]
microwave1 training
Means for microwave1 are
[[   4.24549058]
 [ 145.44526921]
 [  39.70353194]]
car1 training
Not enough samples for car1
oven1 training
Means for oven1 are
[[    3.56979261]
 [ 2197.96146279]
 [  714.80359473]]
waterheater1 training
Not enough samples for waterheater1
dryer1 training
Not enough samples for dryer1
Austin
********************
air1 training
Means for air1 are
[[  3.00917924e-01]
 [  1.88513596e+03]
 [  4.19637327e+02]]
refrigerator1 training
Means for refrigerator1 are
[[ 110.60030677]
 [ 219.71444707]
 [  32.8040773 ]]
clotheswasher1 t

In [156]:
for city, model_city in model.iteritems():
    new_learnt_models = OrderedDict()
    for appliance, appliance_model in model_city.iteritems():
        startprob, means, covars, transmat = sort_learnt_parameters(
                        appliance_model.startprob_, appliance_model.means_,
                        appliance_model.covars_, appliance_model.transmat_)
        new_learnt_models[appliance] = hmm.GaussianHMM(
                    startprob.size, "full", startprob, transmat)
        new_learnt_models[appliance].means_ = means
        new_learnt_models[appliance].covars_ = covars
        from nilmtk.disaggregate.fhmm_exact import create_combined_hmm
        learnt_model_combined = create_combined_hmm(new_learnt_models)
        from nilmtk.disaggregate.fhmm_exact import FHMM
        f = FHMM()
        f.model = learnt_model_combined
        f.individual = new_learnt_models
        import pickle
        pickle.dump(f, open( "../data/model/fhmm_model_%s.p" %city, "wb" ))

In [10]:
out_overall = pickle.load(open('../data/input/all_regions.pkl','r'))

In [11]:
count = 0
for test_region in ["Austin"]:
    out = {}
    test_df = out_overall[test_region]
    test_df=test_df[(test_df.full_agg_available==1)&(test_df.md_available==1)]
    for appliance in ["dw",'hvac','fridge','wm','mw','oven','hvac']:
        if appliance=="hvac":
            start, stop=5, 11
        else:
            start, stop=1, 13
        out[appliance] = {}
        test_df_appliance=test_df.ix[test_df[['%s_%d' %(appliance,month) for month in range(start,stop)]].dropna().index]
        for test_home in test_df_appliance.index:
            pred_df = pd.read_csv("/Users/nipunbatra/wiki-15min-fhmm-out/%s_%d.csv" %(test_region, test_home),index_col=0)
            pred_df_appliance = pred_df[appliance].ix[start:stop-1]
            gt_df_appliance = test_df.ix[test_home][['%s_%d' %(appliance, month) for month in range(start, stop)]]
            gt_df_appliance.index = pred_df_appliance.index
            error = ((pred_df_appliance-gt_df_appliance).abs()*100/gt_df_appliance)
            #error[error>100]=100
            #acc = 100-error
            out[appliance][test_home]=error

In [26]:
pd.DataFrame(out['hvac']).T.median().mean()

30.132167929512207

In [67]:
pd.DataFrame(out).mean()

dw         0.068523
fridge    55.930444
hvac      63.158683
mw         0.945858
oven       0.254512
wm         1.166894
dtype: float64

In [49]:
gt_df_appliance

5     131.292389
6     321.332153
7     628.271118
8     554.701355
9     396.239685
10    171.740158
Name: 9982, dtype: float64

In [146]:
out = {}

for train_region in ["SanDiego","Austin","Boulder"]:
    
    out[train_region] = {}
    train_df = out_overall[train_region]
    for appliance in ["dw",'hvac','fridge','wm','mw','ec','wh','oven']:
        print train_region, appliance
        if appliance=="hvac":
            month_start, month_end = 5, 11
        else:
            month_start, month_end = 1, 13
        out[train_region][appliance] ={}
        for home in train_df.index[1:]:
            try:
                out[train_region][appliance][home] = {}
                pred_df =  pd.read_csv("/Users/nipunbatra/wiki-15min-fhmm-out/%s_%d.csv" %(train_region, home), index_col=0)

                   
                for month in range(month_start, month_end):
                   
                    gt = train_df.ix[home]["%s_%d" %(appliance, month)]
                    pred = pred_df.loc[month,appliance]
                    error = np.abs(gt-pred)
                    error_percentage = 100*error/gt
                    if error_percentage>100:
                        acc=0
                    else:
                        acc = 100-error_percentage
                    out[train_region][appliance][home][month] = acc
            except:
                pass

SanDiego dw
SanDiego hvac
SanDiego fridge
SanDiego wm
SanDiego mw
SanDiego ec
SanDiego wh
SanDiego oven
Austin dw
Austin hvac
Austin fridge
Austin wm
Austin mw
Austin ec
Austin wh
Austin oven
Boulder dw
Boulder hvac
Boulder fridge
Boulder wm
Boulder mw
Boulder ec
Boulder wh
Boulder oven


In [141]:
pd.DataFrame(out['SanDiego']['hvac']).mean().mean()

42.2544186457132

In [142]:
pd.DataFrame(out['SanDiego']['dw']).mean().mean()

0.5577280933464502

In [151]:
pd.DataFrame(out['Austin']['oven']).mean().mean()

0.44467711946446686

In [152]:
pd.DataFrame(out['Boulder']['oven']).mean().mean()

1.2656249398860409

In [155]:
pd.DataFrame(out['Austin']['hvac']).mean().mean()

60.423992969632025

In [158]:
sd_sd_nilm = {}
for appliance in ["dw",'hvac','fridge','wm','mw','ec','wh','oven']:
    sd_sd_nilm[appliance] = pd.DataFrame(out['SanDiego'][appliance]).mean().mean()

In [159]:
sd_sd_nilm

{'dw': 0.5577280933464502,
 'ec': nan,
 'fridge': 50.985555403149746,
 'hvac': 42.2544186457132,
 'mw': 3.681773127817756,
 'oven': 7.0739244574788955,
 'wh': nan,
 'wm': 1.1108455368999053}

In [161]:
pd.DataFrame(out['SanDiego']['hvac'])

,203,527,1450,1524,1731,2031,2354,2606,3687,3864,...,8342,8574,8733,8886,9213,9370,9585,9612,9775,9836
5,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,0.000000,NaN,0.000000,NaN,0,NaN,NaN,20.394375,0.000000,0
6,92.137173,NaN,84.614866,84.682535,46.393935,NaN,NaN,76.633997,54.549238,0.000000,...,0.000000,NaN,0.000000,NaN,0,NaN,NaN,0.000000,0.000000,0
7,70.763907,NaN,93.087525,83.351962,88.188977,NaN,NaN,92.201126,96.582655,80.035744,...,0.000000,NaN,90.715851,NaN,NaN,NaN,NaN,NaN,0.000000,0
8,61.540052,NaN,84.234883,72.917393,76.960231,NaN,NaN,77.641166,82.524598,64.563849,...,85.586980,NaN,71.759366,NaN,NaN,NaN,NaN,NaN,87.398325,NaN
9,72.414384,NaN,80.858820,71.973663,76.534135,NaN,NaN,83.082583,57.828430,57.569448,...,36.116643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.811208,NaN
10,96.714561,NaN,92.523013,95.358362,95.840624,NaN,NaN,92.537795,NaN,71.594192,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.287267,NaN


In [162]:
austin_austin_nilm = {}
for appliance in ["dw",'hvac','fridge','wm','mw','ec','wh','oven']:
    austin_austin_nilm[appliance] = pd.DataFrame(out['Austin'][appliance]).mean().mean()

In [163]:
austin_austin_nilm

{'dw': 0.2785754646785667,
 'ec': nan,
 'fridge': 53.15558262517974,
 'hvac': 60.423992969632025,
 'mw': 3.3594100938387794,
 'oven': 0.44467711946446686,
 'wh': nan,
 'wm': 1.3666039865794135}